In [1]:
import pandas as pd
import re
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
import unicodedata

## 1. Data Acquisition

**Data Citation:**
```
@article{asai2023selfrag,
  author    = {Asai, Akari and Wu, Zeqiu and Wang, Yizhong and Sil, Avirup and Hajishirzi, Hannaneh},
  title     = {{Self-RAG}: Learning to Retrieve, Generate, and Critique through Self-Reflection},
  year      = {2023},
  journal   = { arXiv preprint arXiv:2310.11511 },
  URL       = {https://arxiv.org/abs/2310.11511}
}
```


In [ ]:
# Load data
raw_df = pd.read_json("hf://datasets/selfrag/selfrag_train_data/train.jsonl", lines=True)
raw_df.head()

,instruction,output,input,id,dataset_name
0,"In this task, you are given a context paragrap...",[Retrieval]<paragraph>2017 Portland train atta...,,flan_v2_88425,flan_v2
1,"Question: Write a text based on ""rangers show ...",[No Retrieval]six opposition candidates in the...,,flan_v2_18667,flan_v2
2,You will be given a sentence. Check whether th...,[No Retrieval]1\n****\n[Utility:5],,flan_v2_87754,flan_v2
3,Q:Is there a negative or positive tone to this...,[No Retrieval]Negative[Utility:5],,flan_v2_47789,flan_v2
4,Question: Fertility Clinics Vary on Embryo Dis...,[No Retrieval]Answer: World[Utility:5],,flan_v2_10803,flan_v2


In [ ]:
def check_retrieval(output) -> str:
  """Function to denote if retrieval took place when generating an output"""
  if re.search(r"\[Retrieval\]", output):
    return "true"
  elif re.search(r"\[No Retrieval\]", output):
    return "false"
  else:
    # Some outputs are tagged with `[Continue to Use Evidence]` and `Dialog`.
    # We could flag these as false but we will ignore them instead.
    return "null"

In [ ]:
# Apply check retrieval function
raw_df["retrieval"] = raw_df["output"].apply(check_retrieval)

In [ ]:
# Check for balance
raw_df["retrieval"].value_counts()

retrieval
true     74219
false    70362
null      1038
Name: count, dtype: int64

In [ ]:
# Remove unwanted rows
raw_df.drop(raw_df.loc[raw_df["retrieval"] == "null"].index, inplace=True)
raw_df["retrieval"].value_counts()

retrieval
true     74219
false    70362
Name: count, dtype: int64

## 2. Text Cleaning and Preprocessing

In [ ]:
nlp = spacy.load("en_core_web_sm", disable=["ents", "ner", "parser", "pos_"])

In [ ]:
def clean_preprocess(texts):
    with nlp.select_pipes(enable=["tokenizer", "lemmatizer", "attribute_ruler", "tagger"]):
        for doc in nlp.pipe(texts, batch_size=1000, n_process=2):
            tokens = [
                token.lemma_.lower() for token in doc if
                token.is_ascii and
                not token.is_stop and
                not token.is_punct and
                not token.like_email and
                not token.is_space
                ]
            yield " ".join(tokens)

In [ ]:
raw_df["cleaned_instruction"] = list(clean_preprocess(raw_df["instruction"].values))
raw_df.head()

,instruction,output,input,id,dataset_name,retrieval,cleaned_instruction
0,"In this task, you are given a context paragrap...",[Retrieval]<paragraph>2017 Portland train atta...,,flan_v2_88425,flan_v2,true,task given context paragraph tweet question ta...
1,"Question: Write a text based on ""rangers show ...",[No Retrieval]six opposition candidates in the...,,flan_v2_18667,flan_v2,false,question write text based rangers iron tie ser...
2,You will be given a sentence. Check whether th...,[No Retrieval]1\n****\n[Utility:5],,flan_v2_87754,flan_v2,false,given sentence check sentence grammatically co...
3,Q:Is there a negative or positive tone to this...,[No Retrieval]Negative[Utility:5],,flan_v2_47789,flan_v2,false,q negative positive tone product review = = = ...
4,Question: Fertility Clinics Vary on Embryo Dis...,[No Retrieval]Answer: World[Utility:5],,flan_v2_10803,flan_v2,false,question fertility clinics vary embryo disposa...


In [ ]:
raw_df.to_csv("cleaned_df.csv", columns=["cleaned_instruction", "retrieval"], index=False)

## 3. Feature Engineering

In [2]:
clean_df = pd.read_csv("https://github.com/Nate-Cheney/CIS-405-RAG_Classifier/raw/refs/heads/main/data/cleaned_df.csv")
clean_df.head()

,cleaned_instruction,retrieval
0,task given context paragraph tweet question ta...,True
1,question write text based rangers iron tie ser...,False
2,given sentence check sentence grammatically co...,False
3,q negative positive tone product review = = = ...,False
4,question fertility clinics vary embryo disposa...,False
